In [2]:
from chat_ui import ChatWidget
import pandas as pd
import time

# Create the widget instance
chat = ChatWidget()

# Define a custom message handler
def custom_handler(widget, msg, buffers):
    if msg.lower() == "show sql syntax error":
        # SQL with syntax error
        invalid_query = """
        SELECT name, salary FROM employees
        WHERE department = 'Sales
        ORDER BY salary DESC
        """
        
        # Create the SQL artifact with error
        widget.create_sql_artifact(
            artifact_id="sql_syntax_error",
            query=invalid_query,
            error="Syntax error in SQL statement: Unclosed string literal at line 3",
            title="SQL Query with Syntax Error"
        )
        
        # Send a response message
        widget.send({
            "type": "chat_message", 
            "content": "I've created a SQL artifact with a syntax error. Check the artifact panel to see how errors are displayed."
        })
        
    elif msg.lower() == "show sql missing table":
        # SQL with missing table error
        missing_table_query = """
        SELECT * FROM nonexistent_table
        WHERE id > 10
        """
        
        # Create SQL artifact with missing table error
        widget.create_sql_artifact(
            artifact_id="sql_missing_table",
            query=missing_table_query,
            error="Error: Table 'nonexistent_table' does not exist",
            title="SQL Query with Missing Table Error"
        )
        
        # Send a response message
        widget.send({
            "type": "chat_message", 
            "content": "I've created a SQL artifact with a missing table error. You can see how this type of error is displayed in the artifact panel."
        })
        
    elif msg.lower() == "show sql with result":
        # First, create sample data
        data = {
            'id': [1, 2, 3, 4, 5],
            'name': ['Alice', 'Bob', 'Charlie', 'David', 'Eve'],
            'department': ['Sales', 'Engineering', 'Marketing', 'Sales', 'HR'],
            'salary': [75000, 85000, 70000, 82000, 68000]
        }
        df = pd.DataFrame(data)
        
        # Valid SQL query
        valid_query = """
        SELECT name, department, salary 
        FROM employees
        WHERE department = 'Sales'
        ORDER BY salary DESC
        """
        
        # Simulate query result (in a real scenario, this would come from executing the query)
        result_df = df[df['department'] == 'Sales'].sort_values('salary', ascending=False)
        
        # Create SQL artifact with successful result
        widget.create_sql_artifact(
            artifact_id="sql_valid_query",
            query=valid_query,
            result=result_df,
            title="SQL Query with Results"
        )
        
        # Send a response message
        widget.send({
            "type": "chat_message", 
            "content": "I've created a SQL artifact with a successful query result. You can see the query and its results in the artifact panel."
        })
        
    elif msg.lower() == "show thinking with sql":
        # Demonstrate structured thinking with SQL analysis
        widget.start_thinking()
        
        # Step 1: Understand the query
        widget.add_thinking_step(
            "Analyzing SQL Query", 
            """Let me examine the SQL query:
            ```sql
            SELECT department, AVG(salary) as avg_salary
            FROM employees
            GROUP BY department
            HAVING COUNT(*) > 1
            ```
            
            This query is trying to find the average salary for each department, but only for departments with more than one employee."""
        )
        time.sleep(1.5)
        
        # Step 2: Identify the issue
        widget.add_thinking_step(
            "Identifying SQL Error",
            """When executing this query, I notice an error:

            Error: Syntax error in SQL statement: Invalid column name 'salary' at line 1
            
            The issue appears to be that the 'salary' column doesn't exist in the database schema. Let me check the available columns."""
        )
        time.sleep(1.8)
        
        # Step 3: Inspect schema
        widget.add_thinking_step(
            "Inspecting Database Schema",
            """After examining the database schema, I found that the correct column name is 'annual_salary', not 'salary'.
            
            The corrected query should be:
            ```sql
            SELECT department, AVG(annual_salary) as avg_salary
            FROM employees
            GROUP BY department
            HAVING COUNT(*) > 1
            ```"""
        )
        time.sleep(1.5)
        
        # End thinking process
        widget.end_thinking()
        
        # Create the corrected SQL artifact
        corrected_query = """
        SELECT department, AVG(annual_salary) as avg_salary
        FROM employees
        GROUP BY department
        HAVING COUNT(*) > 1
        """
        
        # Sample result
        result_data = {
            'department': ['Sales', 'Engineering', 'Marketing'],
            'avg_salary': [78500, 92000, 71500]
        }
        result_df = pd.DataFrame(result_data)
        
        widget.create_sql_artifact(
            artifact_id="corrected_sql_query",
            query=corrected_query,
            result=result_df,
            title="Corrected SQL Query"
        )
        
        # Send a response message
        widget.send({
            "type": "chat_message", 
            "content": "I've analyzed the SQL query, found and fixed the error. You can see my thinking process and the corrected query with results in the artifacts panel."
        })
        
    else:
        # For any other message, use the default handler
        widget._default_handle_message(widget, msg, buffers)

# Set the custom message handler
chat.handle_message = custom_handler

# Create welcome artifact with instructions
chat.create_artifact(
    "welcome_artifact",
    """# SQL Artifact Testing

Try these commands to test SQL artifacts:

- Type "show sql syntax error" to see a SQL artifact with syntax error
- Type "show sql missing table" to see a SQL artifact with missing table error
- Type "show sql with result" to see a SQL artifact with successful results
- Type "show thinking with sql" to see structured thinking with SQL error analysis

Each command will create a different SQL artifact that you can view in the artifacts panel.
""",
    "markdown",
    "SQL Artifact Testing Instructions",
    "code"
)

# Display the chat widget
chat

ChatWidget(artifacts={'welcome_artifact': {'id': 'welcome_artifact', 'content': '# SQL Artifact Testing\n\nTry…